In [1]:
!nvidia-smi

Fri Jul 30 16:50:32 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.42.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   54C    P8     8W /  75W |      0MiB /  7611MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [7]:
# Import Keras
from keras.preprocessing import sequence
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM

from keras.optimizers import Adam
from keras.models import load_model
from keras.callbacks import ModelCheckpoint, EarlyStopping

import os
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt

In [3]:
def load_X(path):
    X_signals = []
    files = os.listdir(path)
    files.sort(key=str.lower)
    #['train_acc_x.txt', 'train_acc_y.txt', 'train_acc_z.txt', 'train_gyr_x.txt', 'train_gyr_y.txt', 'train_gyr_z.txt']
    for my_file in files:
        fileName = os.path.join(path,my_file)
        file = open(fileName, 'r')
        X_signals.append(
            [np.array(cell, dtype=np.float32) for cell in [
                row.strip().split(' ') for row in file
            ]]
        )
        file.close()
        #X_signals = 6*totalStepNum*128
    X_signals = np.transpose(np.array(X_signals), (1, 0, 2))#(totalStepNum*6*128)
    return X_signals.reshape(-1,6,128,1)#(totalStepNum*6*128*1)

def load_y(y_path):
    file = open(y_path, 'r')
    # Read dataset from disk, dealing with text file's syntax
    y_ = np.array(
        [elem for elem in [
            row.replace('  ', ' ').strip().split(' ') for row in file
        ]],
        dtype=np.int32
    )
    file.close()
    # Substract 1 to each output class for friendly 0-based indexing
    y_ = y_ - 1
    #one_hot
    y_ = y_.reshape(len(y_))
    n_values = int(np.max(y_)) + 1
    return np.eye(n_values)[np.array(y_, dtype=np.int32)]  # Returns FLOATS


def weight_variable(shape):
    initial = tf.random.truncated_normal(shape, std_dev=0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)


X_train = load_X('/content/drive/Shareddrives/MMH/TIFS20/Dataset #1/train/Inertial Signals')
X_test = load_X('/content/drive/Shareddrives/MMH/TIFS20/Dataset #1/test/Inertial Signals')
train_label = load_y('/content/drive/Shareddrives/MMH/TIFS20/Dataset #1/train/y_train.txt')
test_label = load_y('/content/drive/Shareddrives/MMH/TIFS20/Dataset #1/test/y_test.txt')
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], X_train.shape[2]))
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], X_test.shape[2]))

batch_size = 512
num_classes = 118

In [4]:
X_train.shape, train_label.shape, X_test.shape, test_label.shape

((33104, 6, 128), (33104, 118), (3740, 6, 128), (3740, 118))

# LSTM:
Reference: https://www.analyticsvidhya.com/blog/2019/01/introduction-time-series-classification/

In [5]:
model = Sequential()
model.add(LSTM(128, input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(Dense(118, activation='softmax'))
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 128)               131584    
_________________________________________________________________
dense (Dense)                (None, 118)               15222     
Total params: 146,806
Trainable params: 146,806
Non-trainable params: 0
_________________________________________________________________


In [8]:
adam = Adam(learning_rate=0.001)
chk = ModelCheckpoint('best_model.pkl', monitor='val_accuracy', save_best_only=True, mode='max', verbose=1)
model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])
model.fit(X_train, train_label, validation_split = 0.1, epochs=200, callbacks=[chk, EarlyStopping(monitor='val_loss', patience=30)])

Epoch 1/200
932/932 [==============================] - 23s 6ms/step - loss: 2.3427 - accuracy: 0.5412 - val_loss: 0.5800 - val_accuracy: 0.8644

Epoch 00001: val_accuracy improved from -inf to 0.86439, saving model to best_model.pkl


INFO:tensorflow:Assets written to: best_model.pkl/assets


INFO:tensorflow:Assets written to: best_model.pkl/assets


Epoch 2/200
932/932 [==============================] - 5s 5ms/step - loss: 0.4358 - accuracy: 0.9062 - val_loss: 0.4289 - val_accuracy: 0.8943

Epoch 00002: val_accuracy improved from 0.86439 to 0.89429, saving model to best_model.pkl


INFO:tensorflow:Assets written to: best_model.pkl/assets


...

INFO:tensorflow:Assets written to: best_model.pkl/assets


INFO:tensorflow:Assets written to: best_model.pkl/assets


Epoch 34/200
932/932 [==============================] - 4s 5ms/step - loss: 0.0107 - accuracy: 0.9975 - val_loss: 0.4172 - val_accuracy: 0.9200

Epoch 00034: val_accuracy did not improve from 0.92057
Epoch 35/200
932/932 [==============================] - 4s 4ms/step - loss: 0.0120 - accuracy: 0.9975 - val_loss: 0.4230 - val_accuracy: 0.9160

Epoch 00035: val_accuracy did not improve from 0.92057
Epoch 36/200
932/932 [==============================] - 4s 4ms/step - loss: 0.0126 - accuracy: 0.9963 - val_loss: 0.4366 - val_accuracy: 0.9154

Epoch 00036: val_accuracy did not improve from 0.92057
Epoch 37/200
932/932 [==============================] - 4s 4ms/step - loss: 0.0103 - accuracy: 0.9979 - val_loss: 0.4583 - val_accuracy: 0.9130

Epoch 00037: val_accuracy did not improve from 0.92057
Epoch 38/200
932/932 [==============================] - 4s 4ms/step - loss: 0.0096 - accuracy: 0.9978 - val_loss: 0.4537 - val_accuracy: 0.9142

Epoch 00038: val_accuracy did not improve from 0.92057


INFO:tensorflow:Assets written to: best_model.pkl/assets


INFO:tensorflow:Assets written to: best_model.pkl/assets
